In [1]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

# Assign the database to a variable
db = client.NonRelProj 

# Take the collection "Reviews"
reviews = db.Reviews

# Take the collection "MetaData"
meta = db.MetaData

# Create Indexes

Let's create indexes on our data collections for query efficiancy purposes.

Indexe for reviews:

In [2]:
# Create index on reviews
reviews.create_index(
    {'rating':-1,'num_of_reviews':-1}
)

'rating_-1_num_of_reviews_-1'

### Indexes on restaurants' meta data:

In [3]:
# Create index on average rating
meta.create_index(
    {'name':1, 'avg_rating': -1}
)

'name_1_avg_rating_-1'

We want to create an index based on restaurants' position to enable queries to filter them based on such data.

In [4]:
# Update the schema to store the location as GeoJSON
meta.update_many(
    {},
    [
        { 
            "$set": {
                "location": {
                    "type": "Point",
                    "coordinates": [
                        { "$toDouble": "$longitude" },
                        { "$toDouble": "$latitude" }
                    ]
                }
            }
        }
    ]
)

UpdateResult({'n': 3725, 'nModified': 3725, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Once the location field is set up, create the 2dsphere index

In [5]:
meta.create_index([("location", "2dsphere")])

'location_2dsphere'

In [6]:
# Check whether the 2dsphere index has been created correctly
print(meta.index_information())


{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'name_1_avg_rating_-1': {'v': 2, 'key': [('name', 1), ('avg_rating', -1)]}, 'location_2dsphere': {'v': 2, 'key': [('location', '2dsphere')], '2dsphereIndexVersion': 3}}


In [7]:
# Find documents that have the 'location' field
documents_with_location = meta.find({"location": {"$exists": True}})

# Print the first 1 documents with 'location' to check the data
for doc in documents_with_location.limit(1):
    print(doc)

{'_id': ObjectId('67c751ac8c4ae1c3ac1bceda'), 'name': 'Hale Pops', 'address': 'Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762', 'gmap_id': '0x7c00456eecad3111:0x8217f9600c51f33', 'description': None, 'latitude': 21.637795699999998, 'longitude': -157.9207142, 'category': ['Restaurant'], 'avg_rating': 4.4, 'num_of_reviews': 18, 'price': None, 'hours': [['Thursday', '11AM–8PM'], ['Friday', '11AM–8PM'], ['Saturday', '11AM–8PM'], ['Sunday', 'Closed'], ['Monday', '11AM–8PM'], ['Tuesday', '11AM–8PM'], ['Wednesday', '11AM–8PM']], 'MISC': {'Service options': ['Outdoor seating', 'Takeout', 'Delivery'], 'Popular for': ['Lunch', 'Solo dining'], 'Accessibility': ['Wheelchair accessible entrance'], 'Offerings': ['Comfort food', 'Quick bite'], 'Amenities': ['Good for kids'], 'Atmosphere': ['Casual'], 'Crowd': ['Groups', 'Tourists'], 'Payments': ['NFC mobile payments']}, 'state': 'Closed ⋅ Opens 11AM', 'location': {'type': 'Point', 'coordinates': [-157.9207142, 21.637795699999998]}}
